# Feature Extraction with USB camera 

These modules are necessary to extract features with a USB camera, and display it in Jupyter Notebook.

In [1]:
#Import modules

import matplotlib.pyplot as plt
import cv2
from time import sleep
import numpy as np

To extract a photo from the camera to analyse it, we can use the following function.

In [2]:
#Basic camera functions

def take_photo():
    ret, frame = cap.read()
    time.sleep(2) # wait for the camera to warm up
    cv2.imwrite('webcamphoto.jpg', frame)

The following methods can be use to detect keypoints and descriptors with the camera image.

In [3]:
#Keypoints and descriptors detection methods

def sift(img):    
    # Initialize the object
    sift = cv2.SIFT_create()
    kp, desc = sift.detectAndCompute(imgGray,None)
    return kp, desc

def surf(img):    
    # Initialize the object
    surf = cv2.xfeatures2d.SURF_create() # Not disponible, patent
    surf.hessianThreshold = 500
    kp, desc = surf.detectAndCompute(imgGray,None)
    return kp, desc

def orb(img):
    # Initialize the object
    orb = cv2.ORB_create()
    kp, desc = orb.detectAndCompute(img, None) #img is the gray level image
    print("Number of keypoints Detected: ",len(kp))
    return kp, desc


#Keypoints drawing
def display_kp(img, kp):
    img = cv2.drawKeypoints(img, kp, 0, (255, 0, 0), flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
    return img

To detect objects with the drone camera, we could use the following matches functions.

In [4]:
#Matches detection and display function

def find_matches(desc1, desc2):
    bf = cv2.BFMatcher(cv2.NORM_L2,crossCheck = True)
    matches = bf.match(desc1, desc2)
    matches = sorted(matches, key = lambda x:x.distance)    
    return matches

def display_matches(img1, img2, kp1, kp2, matches):
    img = cv2.drawMatches(img1, kp1, img2, kp2, matches[:10], None, flags = 2)
    return img

We chose the ORB method to detect and compute keypoints and descriptors from video camera. The following function opens a window and displays the keypoints on the image. To close the window, and release the camera access, hit the "q" key on keyboard.

In [5]:
cap = cv2.VideoCapture(2)

while cap.isOpened():
    ret, frame = cap.read()
    #frame2 = frame
    
    imgGray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #imgGray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    kp_orb, desc_orb = orb(imgGray)
    #kp2_orb, desc2_orb = orb(imgGray2)
    
    img2 = display_kp(frame, kp_orb)
    cv2.imshow('Orb', img2)
    #img3 = display_kp(frame2, kp2_orb)
    #cv2.imshow('Orb 2', img3)
    
    #orb_matches = find_matches(desc_orb, desc2_orb)
    #img_orb_matches = display_matches(img2, img3, kp_orb, kp2_orb, orb_matches)
    #cv2.imshow('Orb_matches', img_orb_matches)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Number of keypoints Detected:  0
Number of keypoints Detected:  283
Number of keypoints Detected:  303
Number of keypoints Detected:  346
Number of keypoints Detected:  359
Number of keypoints Detected:  362
Number of keypoints Detected:  381
Number of keypoints Detected:  383
Number of keypoints Detected:  366
Number of keypoints Detected:  388
Number of keypoints Detected:  391
Number of keypoints Detected:  391
Number of keypoints Detected:  392
Number of keypoints Detected:  394
Number of keypoints Detected:  405
Number of keypoints Detected:  410
Number of keypoints Detected:  391
Number of keypoints Detected:  395
Number of keypoints Detected:  397
Number of keypoints Detected:  402
Number of keypoints Detected:  396
Number of keypoints Detected:  391
Number of keypoints Detected:  394
Number of keypoints Detected:  399
Number of keypoints Detected:  413
Number of keypoints Detected:  397
Number of keypoints Detected:  407
Number of keypoints Detected:  408
Number of keypoints De

We want to extract the keypoints and descriptors to an external file.

In [6]:
def pack_features(keypoints, descriptors):
    kpts = np.array([[kp.pt[0], kp.pt[1], kp.size, kp.angle, kp.response, kp.octave, kp.class_id] for kp in keypoints])
    desc = np.array(descriptors)
    return kpts, desc
    
def write_to_npy_file(filename, kp, desc):
    with open('filename', 'wb') as f:
        np.save(filename, np.hstack((kp,desc)))
        
packed_kp, packed_desc = pack_features(kp_orb, desc_orb)
write_to_npy_file('test_save', packed_kp, packed_desc)